In [1]:
import pandas as pd
from tqdm.notebook import tqdm
from os import listdir
from os.path import isfile, join
import shutil
import plotly.express as px
import random
import numpy as np

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
import urllib.request
from requests import get
import re

In [2]:
pathForCharacteristics = 'data/TR824-Digitized/'
pathForCoords ='data/coords/'

## Extracting properties of airfoils

In [3]:
filesInDirectory = [f for f in listdir(pathForCharacteristics) if isfile(join(pathForCharacteristics, f)) and f.split('.')[1]=='ALL']

In [4]:
def extractProperties(filePath, parameter='Alpha, Cl, Cd(interpolated)', num = 1):
    f = open(filePath, "r")
    f = f.read()
    
    dct = {'id':[],'alpha':[],'Cl':[],'Cd':[]}
    for r in f.split(parameter)[1].split('Data set')[1:][num].split('\n')[2:-1]:
        row = [x for x in r.split(' ') if x!='']
        dct['id'].append(filePath.split('/')[-1].split('.')[0])
        dct['alpha'].append(float(row[0]))
        dct['Cl'].append(float(row[1]))
        dct['Cd'].append(float(row[2]))
    return pd.DataFrame(dct)

In [5]:
dfProperties = pd.DataFrame()
for file in filesInDirectory:
    dfProperties = pd.concat([dfProperties, extractProperties(pathForCharacteristics+file)])
dfProperties

,id,alpha,Cl,Cd
0,216,19.37620,1.373150,0.017005
1,216,18.29290,1.411050,0.018944
2,216,17.40130,1.519540,0.000000
3,216,16.28550,1.549260,0.000000
4,216,15.29650,1.531330,0.000000
...,...,...,...,...
24,233,-8.23054,-0.714753,0.010010
25,233,-10.40240,-0.904422,0.000000
26,233,-11.07290,-0.985190,0.000000
27,233,-12.35700,-0.646564,0.009324


## Forming dataframe with ids and airfoil names (type, model)

In [6]:
f = open(pathForCharacteristics+"NACA.LST", "r")
f = f.read()

In [7]:
dct = {'id':[],'type':[],'model':[]}
for r in f.split('---------------------')[1].split('\n')[1:-1]:
    dct['id'].append(r.split('  ')[0].split('.')[0][1:])
    dct['type'].append(r.split('  ')[1])
    dct['model'].append(r.split('  ')[2])
dfNames = pd.DataFrame(dct)
dfNames.head()

,id,type,model
0,131,NACA,0006
1,132,NACA,0009
2,133,NACA,1408
3,134,NACA,1410
4,135,NACA,1412


## Forming a data frame with airfoil geometries

In [8]:
def download(url, file_name):
    response = get(url)
    if response.status_code != 404:
        # open in binary mode
        with open(file_name, "wb") as file:
            # get request
            file.write(response.content)

### Downloading airfoli geometries to coords folder

In [9]:
ids = dfNames['id'].values
for i,r in tqdm(dfNames.iterrows(), total = len(dfNames)):
    mdl = ''.join(c for c in r['model'] if c.isdigit())
    download(f"https://m-selig.ae.illinois.edu/ads/coord/n{r['model']}.dat", 
             pathForCoords+f"{r['id']}.dat")
    download(f"https://m-selig.ae.illinois.edu/ads/coord/naca{r['model']}.dat", 
             pathForCoords+f"{r['id']}.dat")
    download(f"https://m-selig.ae.illinois.edu/ads/coord/naca{mdl}.dat", 
             pathForCoords+f"{r['id']}.dat")

In [10]:
files = [f for f in listdir(pathForCoords) if isfile(join(pathForCoords, f)) and f.split('.')[1]=='dat']
print(f'Succesfully downloaded {len(files)} from {len(dfNames)}')

Succesfully downloaded 64 from 118


### Forming a data frame

In [11]:
dfCoords = pd.DataFrame()
for file in files:
    f = open(pathForCoords+file, "r")
    f = f.read()
    xs = []
    ys = []

    for r in f.split('\n')[1:-1]:
        row = re.findall("-?\d+\.\d+",r)
        if len(row)==2:
            xs.append(float(row[0]))
            ys.append(float(row[1]))
    dfCoords = pd.concat([dfCoords, pd.DataFrame({'id':[file.split('.')[0]],
                                                 'xs':[xs], 
                                                 'ys':[ys]})])

## Joining all together

In [12]:
df2 = dfProperties.merge(dfCoords, left_on='id', right_on='id', how='left')
df2.dropna(inplace=True)

In [13]:
df2.head()

,id,alpha,Cl,Cd,xs,ys
0,216,19.3762,1.37315,0.017005,"[1.0, 0.9502, 0.9004, 0.85055, 0.80063, 0.7506...","[0.0, 0.00744, 0.0166, 0.02649, 0.03653, 0.046..."
1,216,18.2929,1.41105,0.018944,"[1.0, 0.9502, 0.9004, 0.85055, 0.80063, 0.7506...","[0.0, 0.00744, 0.0166, 0.02649, 0.03653, 0.046..."
2,216,17.4013,1.51954,0.000000,"[1.0, 0.9502, 0.9004, 0.85055, 0.80063, 0.7506...","[0.0, 0.00744, 0.0166, 0.02649, 0.03653, 0.046..."
3,216,16.2855,1.54926,0.000000,"[1.0, 0.9502, 0.9004, 0.85055, 0.80063, 0.7506...","[0.0, 0.00744, 0.0166, 0.02649, 0.03653, 0.046..."
4,216,15.2965,1.53133,0.000000,"[1.0, 0.9502, 0.9004, 0.85055, 0.80063, 0.7506...","[0.0, 0.00744, 0.0166, 0.02649, 0.03653, 0.046..."


## Some visuals

In [43]:
idx='216'
sampleAlphaLiftDrag=dfProperties[dfProperties['id']==idx]

xs = dfCoords[dfCoords['id']==idx]['xs'].values[0]
ys = dfCoords[dfCoords['id']==idx]['ys'].values[0]
title=dfNames[dfNames['id']==idx].values[0][1] +' '+\
dfNames[dfNames['id']==idx].values[0][2]
f=px.line(x=xs, y=ys, title='<b>'+title+'</b>')
f.update_yaxes(scaleanchor = "x", scaleratio = 1)
f.show()
f.write_html(f'images/airfoil_{title}')
f = px.line(sampleAlphaLiftDrag, x ='alpha', y='Cl', title=f'Lift coefficient (Cl) vs Angle of attack (alpha) for <b>{title}</b>')
f.show()
f.write_html(f'images/alphaCl_{title}')
f = px.line(sampleAlphaLiftDrag, x ='alpha', y='Cd', title=f'Lift coefficient (Cl) vs Angle of attack (alpha) for <b>{title}</b>')
f.show()
f.write_html(f'images/alphaCd_{title}')

## Feature generation
As soon as x coordinates of shape do not have any valuable information, I decided to find a ratio of y coordinates to x coordinates.  
Later I will need to remove inf values as there are 0's in denominator. This new feature can be considered as a parameter defining angle of a vector connecting the origin with the point and therefore represents geometry of an airfoil. 

In [15]:
df2['xs'] = df2['xs'].apply(lambda x: np.array(x))
df2['ys'] = df2['ys'].apply(lambda x: np.array(x))
df2['ysr'] = df2['ys']/df2['xs']
df2

,id,alpha,Cl,Cd,xs,ys,ysr
0,216,19.37620,1.373150,0.017005,"[1.0, 0.9502, 0.9004, 0.85055, 0.80063, 0.7506...","[0.0, 0.00744, 0.0166, 0.02649, 0.03653, 0.046...","[0.0, 0.00782993054093875, 0.01843625055530875..."
1,216,18.29290,1.411050,0.018944,"[1.0, 0.9502, 0.9004, 0.85055, 0.80063, 0.7506...","[0.0, 0.00744, 0.0166, 0.02649, 0.03653, 0.046...","[0.0, 0.00782993054093875, 0.01843625055530875..."
2,216,17.40130,1.519540,0.000000,"[1.0, 0.9502, 0.9004, 0.85055, 0.80063, 0.7506...","[0.0, 0.00744, 0.0166, 0.02649, 0.03653, 0.046...","[0.0, 0.00782993054093875, 0.01843625055530875..."
3,216,16.28550,1.549260,0.000000,"[1.0, 0.9502, 0.9004, 0.85055, 0.80063, 0.7506...","[0.0, 0.00744, 0.0166, 0.02649, 0.03653, 0.046...","[0.0, 0.00782993054093875, 0.01843625055530875..."
4,216,15.29650,1.531330,0.000000,"[1.0, 0.9502, 0.9004, 0.85055, 0.80063, 0.7506...","[0.0, 0.00744, 0.0166, 0.02649, 0.03653, 0.046...","[0.0, 0.00782993054093875, 0.01843625055530875..."
...,...,...,...,...,...,...,...
3318,233,-8.23054,-0.714753,0.010010,"[1.0, 0.95022, 0.90043, 0.85057, 0.80065, 0.75...","[0.0, 0.00789, 0.0175, 0.02755, 0.03739, 0.046...","[0.0, 0.008303340279093262, 0.0194351587574825..."
3319,233,-10.40240,-0.904422,0.000000,"[1.0, 0.95022, 0.90043, 0.85057, 0.80065, 0.75...","[0.0, 0.00789, 0.0175, 0.02755, 0.03739, 0.046...","[0.0, 0.008303340279093262, 0.0194351587574825..."
3320,233,-11.07290,-0.985190,0.000000,"[1.0, 0.95022, 0.90043, 0.85057, 0.80065, 0.75...","[0.0, 0.00789, 0.0175, 0.02755, 0.03739, 0.046...","[0.0, 0.008303340279093262, 0.0194351587574825..."
3321,233,-12.35700,-0.646564,0.009324,"[1.0, 0.95022, 0.90043, 0.85057, 0.80065, 0.75...","[0.0, 0.00789, 0.0175, 0.02755, 0.03739, 0.046...","[0.0, 0.008303340279093262, 0.0194351587574825..."


## Plotting some random airfoils

In [16]:
df2['ysr'].apply(len).unique()

array([51, 35, 97, 34, 50, 36, 61])

Here is the problem. Each airfoil feometry consists of different number of points. What I decided to do is to shrink the number of points to the lowest number. And remove nan and inf values as well.

In [17]:
df2['ysr'] = df2['ysr'].apply(lambda x: x[~np.isnan(x)]) 
df2['ysr'] = df2['ysr'].apply(lambda x: x[~np.isinf(x)]) 
df2['ysr'].apply(len).unique()

array([50, 34, 96, 33, 49, 35, 60])

In [18]:
def shrink(lst, num = 33):
    while len(lst)!=num:
        lst=np.delete(lst, random.randint(0, len(lst)-1))
    return lst

In [19]:
df2['ysr'] = df2['ysr'].apply(lambda x: shrink(x))

In [20]:
df2[[f'ysr{i}' for i in range(33)]] = pd.DataFrame(df2.ysr.tolist(), index= df2.index)
df2.head()

,id,alpha,Cl,Cd,xs,ys,ysr,ysr0,ysr1,ysr2,...,ysr23,ysr24,ysr25,ysr26,ysr27,ysr28,ysr29,ysr30,ysr31,ysr32
0,216,19.3762,1.37315,0.017005,"[1.0, 0.9502, 0.9004, 0.85055, 0.80063, 0.7506...","[0.0, 0.00744, 0.0166, 0.02649, 0.03653, 0.046...","[0.0, 0.031144553524190228, 0.0617864517418237...",0.00000,0.031145,0.061786,...,-0.160482,-0.140636,-0.121300,-0.102312,-0.067332,-0.051875,-0.038006,-0.015339,-0.006959,-0.001179
1,216,18.2929,1.41105,0.018944,"[1.0, 0.9502, 0.9004, 0.85055, 0.80063, 0.7506...","[0.0, 0.00744, 0.0166, 0.02649, 0.03653, 0.046...","[0.0, 0.00782993054093875, 0.03114455352419022...",0.00000,0.007830,0.031145,...,-0.160482,-0.140636,-0.121300,-0.102312,-0.084171,-0.067332,-0.051875,-0.025783,-0.015339,0.000000
2,216,17.4013,1.51954,0.000000,"[1.0, 0.9502, 0.9004, 0.85055, 0.80063, 0.7506...","[0.0, 0.00744, 0.0166, 0.02649, 0.03653, 0.046...","[0.00782993054093875, 0.031144553524190228, 0....",0.00783,0.031145,0.045627,...,-0.181507,-0.160482,-0.121300,-0.084171,-0.067332,-0.038006,-0.025783,-0.006959,-0.001179,0.000000
3,216,16.2855,1.54926,0.000000,"[1.0, 0.9502, 0.9004, 0.85055, 0.80063, 0.7506...","[0.0, 0.00744, 0.0166, 0.02649, 0.03653, 0.046...","[0.0, 0.00782993054093875, 0.01843625055530875...",0.00000,0.007830,0.018436,...,-0.397185,-0.205310,-0.181507,-0.160482,-0.140636,-0.121300,-0.084171,-0.038006,-0.025783,-0.015339
4,216,15.2965,1.53133,0.000000,"[1.0, 0.9502, 0.9004, 0.85055, 0.80063, 0.7506...","[0.0, 0.00744, 0.0166, 0.02649, 0.03653, 0.046...","[0.00782993054093875, 0.031144553524190228, 0....",0.00783,0.031145,0.045627,...,-0.233617,-0.205310,-0.181507,-0.160482,-0.102312,-0.067332,-0.025783,-0.015339,-0.006959,0.000000


In [21]:
Xy = df2[['alpha']+[f'ysr{i}' for i in range(33)]+['Cl']]
Xy.head()

,alpha,ysr0,ysr1,ysr2,ysr3,ysr4,ysr5,ysr6,ysr7,ysr8,...,ysr24,ysr25,ysr26,ysr27,ysr28,ysr29,ysr30,ysr31,ysr32,Cl
0,19.3762,0.00000,0.031145,0.061786,0.079530,0.098889,0.119679,0.142037,0.189479,0.241246,...,-0.140636,-0.121300,-0.102312,-0.067332,-0.051875,-0.038006,-0.015339,-0.006959,-0.001179,1.37315
1,18.2929,0.00000,0.007830,0.031145,0.061786,0.079530,0.098889,0.119679,0.165420,0.189479,...,-0.140636,-0.121300,-0.102312,-0.084171,-0.067332,-0.051875,-0.025783,-0.015339,0.000000,1.41105
2,17.4013,0.00783,0.031145,0.045627,0.061786,0.079530,0.098889,0.119679,0.142037,0.165420,...,-0.160482,-0.121300,-0.084171,-0.067332,-0.038006,-0.025783,-0.006959,-0.001179,0.000000,1.51954
3,16.2855,0.00000,0.007830,0.018436,0.031145,0.045627,0.098889,0.119679,0.142037,0.165420,...,-0.205310,-0.181507,-0.160482,-0.140636,-0.121300,-0.084171,-0.038006,-0.025783,-0.015339,1.54926
4,15.2965,0.00783,0.031145,0.045627,0.119679,0.142037,0.214482,0.241246,0.271636,0.307773,...,-0.205310,-0.181507,-0.160482,-0.102312,-0.067332,-0.025783,-0.015339,-0.006959,0.000000,1.53133


In [22]:
mms= MinMaxScaler()
X = Xy.iloc[:,:-1]
X = mms.fit_transform(X)
df2 = pd.DataFrame(df2)
rfr = RandomForestRegressor(n_estimators=300)
X_train, X_test, y_train, y_test = train_test_split(X, Xy.iloc[:,-1])

In [23]:
rfr.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=300, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [24]:
print(np.mean(y_train),'±',mean_squared_error(y_train,rfr.predict(X_train), squared=False))
print(np.mean(y_test),'±',mean_squared_error(y_test,rfr.predict(X_test), squared=False))

0.26911524962962974 ± 0.050659977937162134
0.270846811111111 ± 0.1274919619496774


In [25]:
from sklearn.model_selection import cross_val_score
cross_val_score(rfr,X_test, y_test)

array([0.96334855, 0.96213631, 0.96927479, 0.98188037, 0.97481756])

In [26]:
from collections import defaultdict
dct = {'feature':Xy.columns[:-1], 'importance':rfr.feature_importances_}
dfFI = pd.DataFrame(dct)
dfFI.sort_values(by='importance', ascending = False).head(10)

,feature,importance
0,alpha,0.970440
12,ysr11,0.002503
32,ysr31,0.002234
2,ysr1,0.001384
10,ysr9,0.001291
24,ysr23,0.001138
9,ysr8,0.001127
22,ysr21,0.000998
25,ysr24,0.000980
31,ysr30,0.000970


## Randomized Search for Random Forest

In [27]:

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, 
                               param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    8.8s
/opt/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.

[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  5.3min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

In [28]:
rf_random.best_params_

{'n_estimators': 800,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 100,
 'bootstrap': True}

In [29]:
rfr = rf_random.best_estimator_
print(np.mean(y_train),'±',mean_squared_error(y_train,rfr.predict(X_train), squared=False))
print(np.mean(y_test),'±',mean_squared_error(y_test,rfr.predict(X_test), squared=False))

0.26911524962962974 ± 0.05004622405190772
0.270846811111111 ± 0.1266206565569564


## XGBRegressor

In [31]:
from xgboost import XGBRegressor
xgbr = XGBRegressor()
xgbr.fit(X_train, y_train)
print(np.mean(y_train),'±',mean_squared_error(y_train,xgbr.predict(X_train), squared=False))
print(np.mean(y_test),'±',mean_squared_error(y_test,xgbr.predict(X_test), squared=False))

0.26911524962962974 ± 0.008382442788871285
0.270846811111111 ± 0.11827098263123968


## Neural network

In [32]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)


In [37]:

nnModel = Sequential()

nnModel.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))

nnModel.add(Dense(256, kernel_initializer='normal',activation='relu'))
nnModel.add(Dense(256, kernel_initializer='normal',activation='relu'))
nnModel.add(Dense(256, kernel_initializer='normal',activation='relu'))

nnModel.add(Dense(1, kernel_initializer='normal',activation='linear'))

nnModel.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
nnModel.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 128)               4480      
_________________________________________________________________
dense_6 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_7 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_8 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_9 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_10 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_11 (Dense)             (None, 256)              

In [38]:
nnModel.fit(X_train, y_train, epochs=500, batch_size=32, validation_split = 0.2)

Epoch 1/500
34/34 [==============================] - 1s 6ms/step - loss: 0.8294 - mean_absolute_error: 0.8294 - val_loss: 0.7214 - val_mean_absolute_error: 0.7214
Epoch 2/500
34/34 [==============================] - 0s 3ms/step - loss: 0.5534 - mean_absolute_error: 0.5534 - val_loss: 0.4973 - val_mean_absolute_error: 0.4973
Epoch 3/500
34/34 [==============================] - 0s 3ms/step - loss: 0.3060 - mean_absolute_error: 0.3060 - val_loss: 0.2933 - val_mean_absolute_error: 0.2933
Epoch 4/500
34/34 [==============================] - 0s 3ms/step - loss: 0.2059 - mean_absolute_error: 0.2059 - val_loss: 0.1745 - val_mean_absolute_error: 0.1745
Epoch 5/500
34/34 [==============================] - 0s 3ms/step - loss: 0.1903 - mean_absolute_error: 0.1903 - val_loss: 0.2806 - val_mean_absolute_error: 0.2806
Epoch 6/500
34/34 [==============================] - 0s 3ms/step - loss: 0.2131 - mean_absolute_error: 0.2131 - val_loss: 0.1484 - val_mean_absolute_error: 0.1484
Epoch 7/500
34/34 [===

34/34 [==============================] - 0s 3ms/step - loss: 0.1171 - mean_absolute_error: 0.1171 - val_loss: 0.1422 - val_mean_absolute_error: 0.1422
Epoch 52/500
34/34 [==============================] - 0s 3ms/step - loss: 0.1185 - mean_absolute_error: 0.1185 - val_loss: 0.1148 - val_mean_absolute_error: 0.1148
Epoch 53/500
34/34 [==============================] - 0s 3ms/step - loss: 0.1171 - mean_absolute_error: 0.1171 - val_loss: 0.1208 - val_mean_absolute_error: 0.1208
Epoch 54/500
34/34 [==============================] - 0s 3ms/step - loss: 0.1052 - mean_absolute_error: 0.1052 - val_loss: 0.1158 - val_mean_absolute_error: 0.1158
Epoch 55/500
34/34 [==============================] - 0s 6ms/step - loss: 0.1204 - mean_absolute_error: 0.1204 - val_loss: 0.1170 - val_mean_absolute_error: 0.1170
Epoch 56/500
34/34 [==============================] - 0s 3ms/step - loss: 0.1178 - mean_absolute_error: 0.1178 - val_loss: 0.1423 - val_mean_absolute_error: 0.1423
Epoch 57/500
34/34 [=========

34/34 [==============================] - 0s 3ms/step - loss: 0.1197 - mean_absolute_error: 0.1197 - val_loss: 0.1465 - val_mean_absolute_error: 0.1465
Epoch 102/500
34/34 [==============================] - 0s 3ms/step - loss: 0.1100 - mean_absolute_error: 0.1100 - val_loss: 0.1193 - val_mean_absolute_error: 0.1193
Epoch 103/500
34/34 [==============================] - 0s 3ms/step - loss: 0.1030 - mean_absolute_error: 0.1030 - val_loss: 0.1051 - val_mean_absolute_error: 0.1051
Epoch 104/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0898 - mean_absolute_error: 0.0898 - val_loss: 0.1105 - val_mean_absolute_error: 0.1105
Epoch 105/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0996 - mean_absolute_error: 0.0996 - val_loss: 0.1182 - val_mean_absolute_error: 0.1182
Epoch 106/500
34/34 [==============================] - 0s 3ms/step - loss: 0.1084 - mean_absolute_error: 0.1084 - val_loss: 0.1245 - val_mean_absolute_error: 0.1245
Epoch 107/500
34/34 [===

Epoch 151/500
34/34 [==============================] - 0s 3ms/step - loss: 0.1058 - mean_absolute_error: 0.1058 - val_loss: 0.1462 - val_mean_absolute_error: 0.1462
Epoch 152/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0975 - mean_absolute_error: 0.0975 - val_loss: 0.1112 - val_mean_absolute_error: 0.1112
Epoch 153/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0890 - mean_absolute_error: 0.0890 - val_loss: 0.1232 - val_mean_absolute_error: 0.1232
Epoch 154/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0973 - mean_absolute_error: 0.0973 - val_loss: 0.1519 - val_mean_absolute_error: 0.1519
Epoch 155/500
34/34 [==============================] - 0s 3ms/step - loss: 0.1117 - mean_absolute_error: 0.1117 - val_loss: 0.1158 - val_mean_absolute_error: 0.1158
Epoch 156/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0983 - mean_absolute_error: 0.0983 - val_loss: 0.1089 - val_mean_absolute_error: 0.1089
Epoch 157/

Epoch 201/500
34/34 [==============================] - 0s 3ms/step - loss: 0.1034 - mean_absolute_error: 0.1034 - val_loss: 0.1139 - val_mean_absolute_error: 0.1139
Epoch 202/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0905 - mean_absolute_error: 0.0905 - val_loss: 0.1119 - val_mean_absolute_error: 0.1119
Epoch 203/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0829 - mean_absolute_error: 0.0829 - val_loss: 0.1064 - val_mean_absolute_error: 0.1064
Epoch 204/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0879 - mean_absolute_error: 0.0879 - val_loss: 0.1059 - val_mean_absolute_error: 0.1059
Epoch 205/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0880 - mean_absolute_error: 0.0880 - val_loss: 0.1147 - val_mean_absolute_error: 0.1147
Epoch 206/500
34/34 [==============================] - 0s 6ms/step - loss: 0.0797 - mean_absolute_error: 0.0797 - val_loss: 0.1219 - val_mean_absolute_error: 0.1219
Epoch 207/

Epoch 251/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0836 - mean_absolute_error: 0.0836 - val_loss: 0.1172 - val_mean_absolute_error: 0.1172
Epoch 252/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0738 - mean_absolute_error: 0.0738 - val_loss: 0.1205 - val_mean_absolute_error: 0.1205
Epoch 253/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0860 - mean_absolute_error: 0.0860 - val_loss: 0.1123 - val_mean_absolute_error: 0.1123
Epoch 254/500
34/34 [==============================] - 0s 2ms/step - loss: 0.0830 - mean_absolute_error: 0.0830 - val_loss: 0.1173 - val_mean_absolute_error: 0.1173
Epoch 255/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0967 - mean_absolute_error: 0.0967 - val_loss: 0.1228 - val_mean_absolute_error: 0.1228
Epoch 256/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0911 - mean_absolute_error: 0.0911 - val_loss: 0.1093 - val_mean_absolute_error: 0.1093
Epoch 257/

Epoch 301/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0706 - mean_absolute_error: 0.0706 - val_loss: 0.1180 - val_mean_absolute_error: 0.1180
Epoch 302/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0704 - mean_absolute_error: 0.0704 - val_loss: 0.1114 - val_mean_absolute_error: 0.1114
Epoch 303/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0796 - mean_absolute_error: 0.0796 - val_loss: 0.1105 - val_mean_absolute_error: 0.1105
Epoch 304/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0786 - mean_absolute_error: 0.0786 - val_loss: 0.1156 - val_mean_absolute_error: 0.1156
Epoch 305/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0642 - mean_absolute_error: 0.0642 - val_loss: 0.1270 - val_mean_absolute_error: 0.1270
Epoch 306/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0793 - mean_absolute_error: 0.0793 - val_loss: 0.1033 - val_mean_absolute_error: 0.1033
Epoch 307/

Epoch 351/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0712 - mean_absolute_error: 0.0712 - val_loss: 0.1099 - val_mean_absolute_error: 0.1099
Epoch 352/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0669 - mean_absolute_error: 0.0669 - val_loss: 0.1107 - val_mean_absolute_error: 0.1107
Epoch 353/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0718 - mean_absolute_error: 0.0718 - val_loss: 0.1231 - val_mean_absolute_error: 0.1231
Epoch 354/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0751 - mean_absolute_error: 0.0751 - val_loss: 0.1110 - val_mean_absolute_error: 0.1110
Epoch 355/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0717 - mean_absolute_error: 0.0717 - val_loss: 0.1148 - val_mean_absolute_error: 0.1148
Epoch 356/500
34/34 [==============================] - 0s 5ms/step - loss: 0.0711 - mean_absolute_error: 0.0711 - val_loss: 0.1208 - val_mean_absolute_error: 0.1208
Epoch 357/

Epoch 401/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0725 - mean_absolute_error: 0.0725 - val_loss: 0.1115 - val_mean_absolute_error: 0.1115
Epoch 402/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0661 - mean_absolute_error: 0.0661 - val_loss: 0.1376 - val_mean_absolute_error: 0.1376
Epoch 403/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0749 - mean_absolute_error: 0.0749 - val_loss: 0.1125 - val_mean_absolute_error: 0.1125
Epoch 404/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0621 - mean_absolute_error: 0.0621 - val_loss: 0.1171 - val_mean_absolute_error: 0.1171
Epoch 405/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0689 - mean_absolute_error: 0.0689 - val_loss: 0.1201 - val_mean_absolute_error: 0.1201
Epoch 406/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0666 - mean_absolute_error: 0.0666 - val_loss: 0.1093 - val_mean_absolute_error: 0.1093
Epoch 407/

Epoch 451/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0621 - mean_absolute_error: 0.0621 - val_loss: 0.1322 - val_mean_absolute_error: 0.1322
Epoch 452/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0679 - mean_absolute_error: 0.0679 - val_loss: 0.1113 - val_mean_absolute_error: 0.1113
Epoch 453/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0662 - mean_absolute_error: 0.0662 - val_loss: 0.1162 - val_mean_absolute_error: 0.1162
Epoch 454/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0694 - mean_absolute_error: 0.0694 - val_loss: 0.1128 - val_mean_absolute_error: 0.1128
Epoch 455/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0665 - mean_absolute_error: 0.0665 - val_loss: 0.1054 - val_mean_absolute_error: 0.1054
Epoch 456/500
34/34 [==============================] - 0s 3ms/step - loss: 0.0641 - mean_absolute_error: 0.0641 - val_loss: 0.1144 - val_mean_absolute_error: 0.1144
Epoch 457/

In [39]:
print(np.mean(y_train),'±',mean_squared_error(y_train,NN_model.predict(X_train), squared=False))
print(np.mean(y_test),'±',mean_squared_error(y_test,NN_model.predict(X_test), squared=False))

0.26911524962962974 ± 0.10436253240844469
0.270846811111111 ± 0.15689247185471575


## Conclusions
The RMSE values are enormous for test sets for all types of models which is no surprice as I have only 64 records in total which is too few. 
The best model so far is XGBoost.
What I need to do next is to extend my initial dataset. 